In [2]:
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
from scipy import optimize
import functools

In [8]:
def sigmoid(x):                                 
    return 1 / (1 + np.exp(-x))

def sigmoidGradient(z):
    g=np.zeros(np.shape(z))
    g=np.atleast_2d(sigmoid(z)*(1-sigmoid(z))).T
    return g

def randomInitializeWeights(L_in,L_out):
    W=np.zeros([L_out,L_in+1])
    epsilon_init=0.12
    W=np.random.rand(L_out,L_in+1)*2*epsilon_init-epsilon_init
    return W

def nnCostFunction(input_layer_size,hidden_layer_size,num_labels,X,y,lamda,nn_params):
    theta1=np.array(nn_params[0:hidden_layer_size*(input_layer_size+1)]).reshape(input_layer_size+1,hidden_layer_size).T
    theta2=np.array(nn_params[hidden_layer_size*(input_layer_size+1):]).reshape(hidden_layer_size+1,num_labels).T
    m,n=np.shape(X)
    J=0
    Theta1_grad=np.zeros(np.shape(theta1))
    Theta2_grad=np.zeros(np.shape(theta2))
    X2=np.empty([m,n+1])
    X2[:,0]=1
    X2[:,1:]=X
    X=X2
    K=num_labels
    delt3=np.zeros([num_labels,1])
    
    for ii in range(0,K):
        a2i=sigmoid(np.dot(theta1,X.T))
        m0,m1=np.shape(a2i)
        a2=np.empty([m0+1,m1])
        a2[0,:]=1
        a2[1:,:]=a2i
        a3=sigmoid(np.dot(theta2,a2))
        J = J + 1/m*(np.dot(-1*(y==ii+1).T,np.log(a3[ii,:].T) ) - np.dot( ((1-(y==ii+1)).T),(np.log(1-a3[ii,:])).T))
    #regularization
    J = J + lamda/(2*m)*( np.sum(np.sum(theta1[:,1:]**2)) + np.sum(np.sum(theta2[:,1:]**2)) )
    print(J)
    return J

def nnGradFunction(input_layer_size,hidden_layer_size,num_labels,X,y,lamda,nn_params):
    theta1=np.array(nn_params[0:hidden_layer_size*(input_layer_size+1)]).reshape(input_layer_size+1,hidden_layer_size).T
    theta2=np.array(nn_params[hidden_layer_size*(input_layer_size+1):]).reshape(hidden_layer_size+1,num_labels).T
    m,n=np.shape(X)
    J=0
    Theta1_grad=np.zeros(np.shape(theta1))
    Theta2_grad=np.zeros(np.shape(theta2))
    X2=np.empty([m,n+1])
    X2[:,0]=1
    X2[:,1:]=X
    X=X2
    K=num_labels
    delt3=np.zeros([num_labels,1])
       
    #backward propogation
    d1=0
    d2=0
    for t in range(0,m):
        z2 = np.dot(theta1,X[t,:].T)
        a2i = sigmoid(z2)
        a2i = np.atleast_2d(a2i)
        m2,m3=np.shape(a2i)
        a2=np.ones([m3+1,m2])
        a2[1:,:]=a2i.T
        z3 = np.dot(theta2,a2)
        a3 = sigmoid(z3)
        for i in range(0,num_labels):
            delt3[i] = a3[i] - (y[t]==i+1)
        delt2 = np.dot(theta2[:,1:].T,delt3)*sigmoidGradient(z2)
        d2 = d2+np.dot(delt3,a2.T)
        d1 = d1 + np.dot(delt2,np.atleast_2d(X[t,:]))
    Theta1_grad = d1/m    
    Theta2_grad = d2/m
    #regularization
    Theta1_grad[:,1:] = Theta1_grad[:,1:]+lamda/m*theta1[:,1:]
    Theta2_grad[:,1:] = Theta2_grad[:,1:]+lamda/m*theta2[:,1:]
    
    #unroll gradients
    grad1=Theta1_grad.T.flatten()
    grad2=Theta2_grad.T.flatten()
    grad=np.concatenate((grad1,grad2))
    return grad


In [10]:
input_layer_size=400
hidden_layer_size=25
num_labels=10
contents = sio.loadmat('./machine-learning-ex4/machine-learning-ex4/ex4/ex4data1.mat')
X=contents['X']
y=contents['y']
contents2 = sio.loadmat('./machine-learning-ex4/machine-learning-ex4/ex4/ex4weights.mat')
Theta1=contents2['Theta1']
Theta2=contents2['Theta2']
#unroll parameters
nn_params1=Theta1.T.flatten()
nn_params2=Theta2.T.flatten()
nn_params=np.concatenate((nn_params1,nn_params2))
lamda=1
J=nnCostFunction(input_layer_size,hidden_layer_size,num_labels,X,y,lamda,nn_params)
print(J[0])

[ 0.38376986]
0.383769859091


In [ ]:
#check NN gradients function
def diw(fan_out,fan_in):
    w=np.array(np.sin(list(range(1,fan_out*(fan_in+1)+1))))/10
    return w.reshape([fan_in+1,fan_out]).T


lamdat=3
input_layer_sizet=3
hidden_layer_sizet=5
num_labelst=3
mt=5
Theta1t=diw(hidden_layer_sizet,input_layer_sizet)
Theta2t=diw(num_labelst,hidden_layer_sizet)
Xt=diw(mt,input_layer_sizet-1)
yt=np.atleast_2d(1 + np.mod(range(1,mt+1),num_labelst)).T
# print(Theta1t.shape,Theta2t.shape,Xt.shape,yt.shape)

nn_paramst1=Theta1t.T.flatten()
nn_paramst2=Theta2t.T.flatten()
nn_paramst=np.atleast_2d(np.concatenate((nn_paramst1,nn_paramst2))).T
# print(nn_paramst.shape)

costt,gradt=nnCostFunction(input_layer_sizet,hidden_layer_sizet,num_labelst,Xt,yt,lamdat,nn_paramst)

costFunction=functools.partial(nnCostFunction,input_layer_sizet,hidden_layer_sizet,num_labelst,Xt,yt,lamdat)

#numerical calculation for comparison
def cng(costFunction,theta):
    numgrad=np.zeros(np.shape(theta))
    perturb=np.zeros(np.shape(theta))
    e=1e-4
    for p in range(0,np.size(theta)):
        perturb[p]=e
        loss1=costFunction(theta-perturb)[0]
        loss2=costFunction(theta+perturb)[0]
#         loss1=nnCostFunction(theta-perturb,input_layer_sizet,hidden_layer_sizet,num_labelst,Xt,yt,lamdat)[0]
#         loss2=nnCostFunction(theta+perturb,input_layer_sizet,hidden_layer_sizet,num_labelst,Xt,yt,lamdat)[0]
        numgrad[p]=(np.array(loss2)-np.array(loss1))/(2*e)
        perturb[p]=0
    return numgrad


gradtt=cng(costFunction,nn_paramst)
gradt=np.atleast_2d(gradt).T
print(sum(gradt-gradtt))
print(costt)

In [83]:
%%time
#tests finished, time for training
print('training neural network...')
options=400 #max interations
lamda=2

#initialize random weights
initial_theta1 = randomInitializeWeights(input_layer_size,hidden_layer_size)
initial_theta2 = randomInitializeWeights(hidden_layer_size,num_labels)
#unroll params
initial_params1 = initial_theta1.T.flatten()
initial_params2 = initial_theta2.T.flatten()
initial_params = np.atleast_2d(np.concatenate((initial_params1,initial_params2))).T

CostFunction = functools.partial(nnCostFunction,input_layer_size,hidden_layer_size,num_labels,X,y,lamda)
GradFunction = functools.partial(nnGradFunction,input_layer_size,hidden_layer_size,num_labels,X,y,lamda)

print(type(input_layer_size),type(hidden_layer_size),type(num_labels),X.shape,y.shape,type(lamda),initial_params.shape)
print(CostFunction(initial_params))


counter=0
def callbackOutput(xk):
    global counter
    counter=counter+1
    print('Iteration Number: '+str(counter))
    

output=optimize.fmin_cg(CostFunction,initial_params,fprime=GradFunction,maxiter=400,callback=callbackOutput)

training neural network...
<class 'int'> <class 'int'> <class 'int'> (5000, 400) (5000, 1) <class 'int'> (10285, 1)
[ 7.02675523]
[ 7.02675523]
[ 7.02675523]
[ 4.3437232]
[ 3.31639122]
Iteration Number: 1
[ 3.26143803]
Iteration Number: 2
[ 3.25459897]
[ 3.23317692]
[ 3.21601434]
Iteration Number: 3
[ 3.13191264]
[ 2.97757391]
Iteration Number: 4
[ 2.77460888]
[ 2.7021168]
[ 2.66589903]
Iteration Number: 5
[ 2.28055912]
[ 2.3932444]
Iteration Number: 6
[ 2.1184883]
Iteration Number: 7
[ 1.95759872]
[ 1.93094121]
Iteration Number: 8
[ 1.86340353]
[ 1.7771731]
Iteration Number: 9
[ 1.67468599]
[ 1.54098291]
Iteration Number: 10
[ 1.47382787]
[ 1.43419334]
Iteration Number: 11
[ 1.38539272]
[ 1.29761407]
Iteration Number: 12
[ 1.24933663]
[ 1.19745111]
Iteration Number: 13
[ 1.17570687]
[ 1.14937063]
Iteration Number: 14
[ 1.12474098]
[ 1.0943126]
Iteration Number: 15
[ 1.07420458]
[ 1.03185897]
Iteration Number: 16
[ 1.00647587]
[ 0.97827945]
Iteration Number: 17
[ 0.96245801]
[ 0.941150

In [84]:
%%time
print(np.shape(output))
Theta1=np.array(output[0:hidden_layer_size*(input_layer_size+1)]).reshape(input_layer_size+1,hidden_layer_size).T
Theta2=np.array(output[hidden_layer_size*(input_layer_size+1):]).reshape(hidden_layer_size+1,num_labels).T
print(Theta1.shape)
print(Theta2.shape)
def predict(Theta1,Theata2,X):
    m,n=np.shape(X)
    num_labels=np.shape(Theta2)[0]
    p=np.zeros([m,1])
    X2=np.empty([m,n+1])
    X2[:,0]=1
    X2[:,1:]=X
    X=X2
    h1=sigmoid(np.dot(X,Theta1.T))
    i,j=np.shape(h1)
    h2i=np.ones([i,j+1])
    h2i[:,1:]=h1
    h2=sigmoid(np.dot(h2i,Theta2.T))
    p=np.argmax(h2,1)
    return p
pred = predict(Theta1,Theta2,X)
print(pred.shape)
print(y.shape)
accuracy=np.sum(np.atleast_2d(pred+1).T==y)/5000
print(accuracy)

(10285,)
(25, 401)
(10, 26)
(5000,)
(5000, 1)
0.985
Wall time: 25 ms
